In [1]:
# import the MAAP package to handle queries
from maap.maap import MAAP


# Choose bounding box and other parameters

The bounding box should be a single string of four lat/lon, in the order:
[left  bottom  right top]

In [11]:
bbox = '-118.06817 34.22169 -118.05801 34.22822'  # Mt Wilson (near JPL) (small, ~8 sec)
# bbox = '-124.81360 32.44506 -113.75989 42.24498' # CA and NV (uses ~5-6 GB RAM, takes a few minutes to run)

# Very large bbox options -- might run out of memory
# bbox = '-125.53514 25.42150  -87.06960 49.22105' # Western US (CA, WA, Chicago, New Orleans)
# bbox = '-125.41615 31.03621 -101.80916 49.17324' # Western US (CA -> CO)
# bbox =    '6.26868 36.00380   18.57179 47.28139'  # Italy (Requires more than 8 GB RAM)


# Number of DPS jobs to launch
num_DPS_jobs = 2

# True to perform compute-intensive computations
run_compute = True

## Launch jobs in DPS

If you're still in development, skip this section
and jump to "Test Run in ADE" section below.

It's faster to test the algorithm in the ADE first, and
then run it on DPS.

In [47]:
# Register Algorithm from YAML file
# maap = MAAP()
# maap.register_algorithm_from_yaml_file("/projects/GET-DEM-v0.2.0.yml").text

In [12]:
# Submit DPS Job
maap = MAAP(maap_host='api.maap-project.org')

jobs = []

for job_num in range(num_DPS_jobs):
    jobs.append(
        maap.submitJob(
            identifier="test5",  # create a unique job identifier
            algo_id="GET-DEM-v0.2.0",  # Name of the registered algorithm
            version="develop",
            username="niemoell",
            queue="maap-dps-worker-32vcpu-64gb",
            # for benchmarking consistency, use same bbox for all jobs
            bbox=bbox,
            # TODO: Is there a cleaner design for specifying CLI flags for DPS jobs?
            compute="--compute" if run_compute else ""
        )
    )


In [15]:
# Monitor the job status via JN.
# For a GUI, go to "View and Submit Jobs" via the Launcher
n_failed = 0
n_succeeded = 0
n_running = 0
n_except = 0
n_accepted = 0

for j in jobs:
    try:
        j.retrieve_attributes()
    except:
        n_except += 1
        continue
    if j.status.lower() == 'failed':
        n_failed += 1
    elif j.status.lower() == 'succeeded':
        n_succeeded += 1
    elif j.status.lower() == 'running':
        n_running += 1
    elif j.status.lower() == 'accepted':
        n_accepted += 1
    else:
        print(j.status)

print("failed: ", n_failed)
print("accepted: ", n_accepted)
print("running: ", n_running)
print("succeeded: ", n_succeeded)
print("except: ", n_except)


failed:  0
accepted:  0
running:  0
succeeded:  2
except:  0


In [16]:
print(type(jobs[0]))
print(jobs[0])

<class 'maap.dps.dps_job.DPSJob'>
{'job_id': 'f6733988-ad47-4838-9acb-e1975d625d7c', 'status': 'Succeeded', 'machine_type': None, 'architecture': None, 'machine_memory_size': None, 'directory_size': None, 'operating_system': None, 'job_start_time': None, 'job_end_time': None, 'job_duration_seconds': None, 'cpu_usage': None, 'cache_usage': None, 'mem_usage': None, 'max_mem_usage': None, 'swap_usage': None, 'read_io_stats': None, 'write_io_stats': None, 'sync_io_stats': None, 'async_io_stats': None, 'total_io_stats': None, 'error_details': None, 'response_code': 200, 'outputs': ['http://maap-ops-workspace.s3-website-us-west-2.amazonaws.com/niemoell/dps_output/GET-DEM-v0.2.0/develop/test5/2024/02/02/12/32/23/013949', 's3://s3.us-west-2.amazonaws.com:80/maap-ops-workspace/niemoell/dps_output/GET-DEM-v0.2.0/develop/test5/2024/02/02/12/32/23/013949', 'https://s3.console.aws.amazon.com/s3/buckets/maap-ops-workspace/niemoell/dps_output/GET-DEM-v0.2.0/develop/test5/2024/02/02/12/32/23/013949/?r

## Test Run in the ADE

In [18]:
# To run from the ADE, update the conda environment:
# (For DPS, this step is handled during algorithm registration.)
!./build-env.sh

Channels:
 - conda-forge
Platform: linux-64
Solving environment: done

libaec-1.1.2         | 34 KB     |                                       |   0% 
kealib-1.5.3         | 169 KB    |                                       |   0% 

lerc-4.0.0           | 275 KB    |                                       |   0% 


libnsl-2.0.1         | 33 KB     |                                       |   0% 



libxcrypt-4.4.36     | 98 KB     |                                       |   0% 




krb5-1.21.2          | 1.3 MB    |                                       |   0% 





font-ttf-dejavu-sans | 388 KB    |                                       |   0% 






libcblas-3.9.0       | 14 KB     |                                       |   0% 







pcre2-10.42          | 993 KB    |                                       |   0% 








libgomp-13.2.0       | 410 KB    |                                       |   0% 









libblas-3.9.0        | 14 KB     |                                       |

In [20]:
# WARNING - only use for testing! Not DPS!
import os
from time import time

# Build a DEM
start = time()

# Do a test run via CLI to ensure input arguments are passed correctly to the algorithm.
# !./run.sh -118.06817 34.22169 -118.05801 34.22822  # small bbox; should take 7-10 seconds.

# Hint: Make sure to initialize `bbox` above. Otherwise will error.
!./run.sh {bbox} --compute
# !./run.sh {bbox}


print(f"Time to make DEM: {time()-start} seconds")


[02/02 12:45:02] [INFO dem.py] Bounds: -118.06817 34.22169 -118.05801 34.22822
[02/02 12:45:03] [INFO cop_dem.py] Creating output/dem.tif
[02/02 12:45:03] [INFO cop_dem.py] Fetching remote tiles...
[02/02 12:45:03] [INFO cop_dem.py] Running GDAL command:
[02/02 12:45:03] [INFO cop_dem.py] gdalwarp /vsicurl/https://raw.githubusercontent.com/scottstanie/sardem/master/sardem/data/cop_global.vrt output/dem.tif -of GTiff -ot Int16 -te -118.068169999999995 34.2216900000000024 -118.058009999999996 34.2282200000000003 -tr 0.000277777777777777778 0.000277777777777777778 -s_srs "epsg:4326+3855" -t_srs "epsg:4326" -wo NUM_THREADS=4 -r nearest -wm 5000 -multi
Creating output file that is 37P x 24L.
0...10...20...30...40...50...60...70...80...90...100 - done.
Time to fetch and create dem.tif: 3.78686261177063 seconds
/opt/conda/envs/dem/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, ex